# EPA112A - Programming for Data Science - Group 31

- Georges Puttaert - 4686160
- Thijs Roolvink
- Gijs de Werd

## Research Question

 **Income Inequality and Social Welfare: What is the impact of income inequality on social welfare indicators, such as life expectancy, education access, and healthcare quality for countries in the low, lower middle, upper middle and high income categories?**

Link to countries per income category:
<br>
https://datatopics.worldbank.org/world-development-indicators/the-world-by-income-and-region.html

Chosen Countries per category:
- **Upper income**: United States of America (USA)
- **Upper middle income**: Argentina (ARG)
- **Lower middle income**: India (IND)
- **Low income**: Ethiopia (ETH)

### Packages

In [1]:
import wbdata

### Indicators from the World Bank

In [2]:
# Define indicators
health_indicators = {'SH.XPD.CHEX.GD.ZS': "Health Expenditure as a Percentage of GDP", "SH.IMM.IDPT": "Immunization"}
GDP_indicator = {'NY.GDP.PCAP.CD': 'gdppc'}
life_exp_indicator = {'SP.DYN.LE00.IN': 'Life Expectancy at Birth', 'SH.DYN.MORT': 'Child Mortality'}
immuni_childmortality_indicator = {"SH.IMM.IDPT": "Immunization", 'SH.STA.MMRT': 'Maternal Mortality Rate'}

### DataFrames for chosen indicators and countries from the World Bank

In [3]:
#United States of America
df_health_usa = wbdata.get_dataframe(health_indicators, country=['USA'], convert_date=True)
df_gdp_usa = wbdata.get_dataframe(GDP_indicator, country=['USA'], convert_date=True)
df_life_exp_usa = wbdata.get_dataframe(life_exp_indicator, country=['USA'], convert_date=True)
df_immuni_childmortality_usa = wbdata.get_dataframe(immuni_childmortality_indicator, country=['USA'], convert_date=True)

#Argentina
df_health_arg = wbdata.get_dataframe(health_indicators, country=['ARG'], convert_date=True)
df_gdp_arg = wbdata.get_dataframe(GDP_indicator, country=['ARG'], convert_date=True)
df_life_exp_arg = wbdata.get_dataframe(life_exp_indicator, country=['ARG'], convert_date=True)
df_immuni_childmortality_arg = wbdata.get_dataframe(immuni_childmortality_indicator, country=['ARG'], convert_date=True)

#India
df_health_ind = wbdata.get_dataframe(health_indicators, country=['IND'], convert_date=True)
df_gdp_ind = wbdata.get_dataframe(GDP_indicator, country=['IND'], convert_date=True)
df_life_exp_ind = wbdata.get_dataframe(life_exp_indicator, country=['IND'], convert_date=True)
df_immuni_childmortality_ind = wbdata.get_dataframe(immuni_childmortality_indicator, country=['IND'], convert_date=True)

#Ethiopia
df_health_eth = wbdata.get_dataframe(health_indicators, country=['ETH'], convert_date=True)
df_gdp_eth = wbdata.get_dataframe(GDP_indicator, country=['ETH'], convert_date=True)
df_life_exp_eth = wbdata.get_dataframe(life_exp_indicator, country=['ETH'], convert_date=True)
df_immuni_childmortality_eth = wbdata.get_dataframe(immuni_childmortality_indicator, country=['ETH'], convert_date=True)

KeyboardInterrupt: 

In [4]:
import plotly.graph_objs as go

indicators = {
    'NY.GDP.PCAP.CD': 'GDP per Capita',
    'SI.POV.GINI': 'Income Inequality (Gini Index)',
    'SP.DYN.LE00.IN': 'Life Expectancy',
    'SI.POV.DDAY': 'Poverty headcount',
    'EG.ELC.ACCS.ZS': 'Acces to electricity (percentage of pop)',
    'UIS.XGDP.1.FSGOV': 'Government expenditure on primary education as % of GDP (%)'
}

countries = ['USA', 'ARG', 'IND','ETH']  # Replace with actual country codes
countries = countries[1]

df = wbdata.get_dataframe(indicators, country=countries, convert_date=True)

fig = go.Figure()
fig2 = go.Figure()

for indicator_code, indicator_name in indicators.items():
    if indicator_name == 'GDP per Capita':
        continue
    fig.add_trace(go.Scatter(x=df.index, y=df[indicator_name], mode='lines', name=indicator_name))

fig2.add_trace(go.Scatter(x=df.index, y=df['GDP per Capita'], mode='lines', name='GDP per Capita'))

fig.update_layout(title="Social Welfare Indicators Over Time", xaxis_title="Year", yaxis_title="Value")
fig2.update_layout(title="Social Welfare Indicators Over Time", xaxis_title="Year", yaxis_title="Euros (x1000)")

fig.show()
fig2.show()
